In [1]:
import pandas as pd

In [2]:
#https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/

In [3]:
dataset=pd.read_csv("Airmean_chennai.csv")

In [4]:
dataset

,Date,City,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI
0,2015-01-01,Chennai,77.394417,149.388739,16.30,15.39,22.68,4.59,1.17,9.20,11.35000,0.170000,7.402406,1.197632,170.088957
1,2015-01-02,Chennai,77.394417,149.388739,16.49,13.42,23.09,7.83,1.23,8.61,9.16000,0.130000,7.402406,1.197632,170.088957
2,2015-01-03,Chennai,77.394417,149.388739,9.72,19.56,9.99,4.63,0.77,20.56,13.45000,0.030000,7.402406,1.197632,170.088957
3,2015-01-04,Chennai,77.394417,149.388739,9.60,16.20,11.71,5.23,1.00,20.56,10.33000,2.120333,7.402406,1.197632,170.088957
4,2015-01-05,Chennai,77.394417,149.388739,9.16,16.30,12.94,5.50,0.90,16.60,9.36000,2.120333,7.402406,1.197632,170.088957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,2020-06-27,Chennai,26.420000,83.998141,7.25,12.96,19.59,33.20,1.10,7.29,68.51000,0.100000,0.070000,1.197632,95.000000
2005,2020-06-28,Chennai,25.930000,83.998141,7.81,10.00,16.39,35.98,0.76,6.48,72.40625,0.090000,0.000000,1.197632,98.000000
2006,2020-06-29,Chennai,21.300000,83.998141,7.65,9.69,16.74,34.07,0.96,6.62,62.57000,0.090000,0.010000,1.197632,104.000000
2007,2020-06-30,Chennai,24.140000,83.998141,8.42,12.38,20.29,34.17,1.05,7.50,68.75000,0.170000,0.160000,1.197632,110.000000


In [5]:
dataset["City"].value_counts()

City
Chennai    2009
Name: count, dtype: int64

In [6]:
dataset.isnull().sum()

Date       0
City       0
PM2.5      0
PM10       0
NO         0
NO2        0
NOx        0
NH3        0
CO         0
SO2        0
O3         0
Benzene    0
Toluene    0
Xylene     0
AQI        0
dtype: int64

In [7]:
timedata=pd.DataFrame()

In [8]:
timedata["Date"]=dataset["Date"]
timedata["AQI"]=dataset["AQI"]

In [9]:
timedata

,Date,AQI
0,2015-01-01,170.088957
1,2015-01-02,170.088957
2,2015-01-03,170.088957
3,2015-01-04,170.088957
4,2015-01-05,170.088957
...,...,...
2004,2020-06-27,95.000000
2005,2020-06-28,98.000000
2006,2020-06-29,104.000000
2007,2020-06-30,110.000000


In [10]:
data=timedata["AQI"]

In [11]:
data_train=data.head(1500)

In [12]:
data_test=data.tail(509)

In [13]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(data_train, order=(1,1,1), seasonal_order=(0,0,0,0))#Seasonal Autoregressive Integrated Moving Average (ARIMA)
model_fit = model.fit()
# make prediction
y_pred= model_fit.predict(len(data_train), len(data)-1)
print(y_pred)
from sklearn.metrics import r2_score
r2=r2_score(data_test,y_pred)
print("r2={}".format(r2))

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.04817D+00    |proj g|=  6.16111D-02

At iterate    5    f=  5.02389D+00    |proj g|=  2.03802D-02

At iterate   10    f=  5.01978D+00    |proj g|=  1.91869D-03

At iterate   15    f=  5.01959D+00    |proj g|=  1.25437D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     16     18      1     0     0   3.676D-06   5.020D+00
  F =   5.0195871380236472     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
1500    

 This problem is unconstrained.


In [14]:
trends=['n','t','c','ct']
orders=[(0,0,0),(0,0,1),(2,0,1),(1,1,1)]
orderslist=[]
trendslist=[]
rscorelist=[]
for td in trends:
    for i in orders:
        trendslist.append(td)
        orderslist.append(i)
        from statsmodels.tsa.statespace.sarimax import SARIMAX
        model = SARIMAX(data_train, order=i, seasonal_order=(0, 0, 0, 0), trend=td)
        model_fit = model.fit()
    # make prediction
        y_pred= model_fit.predict(len(data_train), len(data)-1)
        print(y_pred)
        from sklearn.metrics import r2_score
        r2=r2_score(data_test,y_pred)
        rscorelist.append(r2)
        print("Trend={}, Order={} ,r2={}".format(td,i,r2))

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.33443D+00    |proj g|=  5.32907D-10

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    1      0      1      0     0     0   5.329D-10   6.334D+00
  F =   6.3344333762363085     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.
 This problem is unconstrained.


1500    0.0
1501    0.0
1502    0.0
1503    0.0
1504    0.0
       ... 
2004    0.0
2005    0.0
2006    0.0
2007    0.0
2008    0.0
Name: predicted_mean, Length: 509, dtype: float64
Trend=n, Order=(0, 0, 0) ,r2=-6.410263024207206
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.01876D+00    |proj g|=  3.26895D-02

At iterate    5    f=  5.99483D+00    |proj g|=  7.58421D-02

At iterate   10    f=  5.86242D+00    |proj g|=  2.56054D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg       

 This problem is unconstrained.
 This problem is unconstrained.
 This problem is unconstrained.


...


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
 This problem is unconstrained.



At iterate    5    f=  5.02349D+00    |proj g|=  1.31012D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    6      9     16      1     0     0   8.081D-05   5.023D+00
  F =   5.0231644959666015     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
1500    91.730730
1501    91.386137
1502    91.274506
1503    91.177308
1504    91.095855
          ...    
2004    67.649280
2005    67.602543
2006    67.555806
2007    67.509069
2008    67.462332
Name: predicted_mean, Length: 509, dtype: float64
Trend=ct, Order=(2, 0, 1) ,r2=-0.08580689103719541
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision

 This problem is unconstrained.



At iterate   20    f=  5.04160D+00    |proj g|=  9.47343D-01

At iterate   25    f=  5.03145D+00    |proj g|=  1.04610D+00

At iterate   30    f=  5.02179D+00    |proj g|=  2.05379D+01

At iterate   35    f=  5.02153D+00    |proj g|=  6.88934D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     36     61      1     0     0   6.889D-01   5.022D+00
  F =   5.0215270817655933     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
1500    102.906698
1501    112.460225
1502    118.214256
1503    121.696046
1504    123.818984
           ...    
2004    204.585372
2005    204.795677
2006    205.006201
2007


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


In [15]:
rscorelist

[-6.410263024207206,
 -6.405887104312315,
 -0.4405523254539634,
 -0.6459179336827807,
 -9.106679032348639,
 -9.07944091985154,
 -1.3866114872652244,
 -0.3246848587652371,
 -0.8177724908589699,
 -0.8152342890187421,
 -0.7570902117998539,
 -0.22730169896455066,
 -0.07528909496166114,
 -0.04811385980390814,
 -0.08580689103719541,
 -4.13652142221413]

In [16]:
result=pd.DataFrame()

In [17]:
result.insert(0,"Trend",trendslist)
result.insert(1,"Order",orderslist)
result.insert(2,"R_score",rscorelist)

In [18]:
result

,Trend,Order,R_score
0,n,"(0, 0, 0)",-6.410263
1,n,"(0, 0, 1)",-6.405887
2,n,"(2, 0, 1)",-0.440552
3,n,"(1, 1, 1)",-0.645918
4,t,"(0, 0, 0)",-9.106679
5,t,"(0, 0, 1)",-9.079441
6,t,"(2, 0, 1)",-1.386611
7,t,"(1, 1, 1)",-0.324685
8,c,"(0, 0, 0)",-0.817772
9,c,"(0, 0, 1)",-0.815234


In [19]:
model_fit.predict(len(data), len(data)+5)

2009    205.639086
2010    205.850485
2011    206.062102
2012    206.273939
2013    206.485994
2014    206.698268
Name: predicted_mean, dtype: float64